In [1]:
import os
import sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
sys.path.insert(1, r'C:\Users\smartin5\Repos\myNeurochat')
#sys.path.insert(1, r'C:\Users\maolivei\neuro_sean\NeuroChaT')
%load_ext autoreload
%autoreload 2

In [2]:
from neurochat.nc_spike import NSpike
from neurochat.nc_spatial import NSpatial
from neurochat.nc_data import NData
from neurochat.nc_datacontainer import NDataContainer
import neurochat.nc_plot as nc_plot
import neurochat.nc_containeranalysis as nca
from neurochat.nc_clust import NClust
from neurochat.nc_utils import butter_filter
from neurochat.nc_utils import find_peaks

import numpy as np

In [3]:
# Setup the main data path
home = os.path.expanduser("~")
data_dir = os.path.join(
    home, "Recordings", "LCA1")
prepend_data_dir = lambda name : os.path.join(data_dir, name)

In [4]:
# Load the data
"""
spike_filenames = [
    "LCA1-linearpresleep30min050319.2",
    "050319D_LCA1_linear_10min.2",
    "LCA1-linearpresleep_40min050319_B.2"]
"""

spike_filenames = [
#    "LCA1-linearpresleep30min060319.2",
    "060319D_LCA1_linear_15min.2",
    "060319E_LCA1_linear_sleep_32min.2"]

spike_absolute_filenames = [prepend_data_dir(name) for name in spike_filenames]

spike_names = [
#    "pre-sleep",
    "run",
    "post-sleep"]

position_absolute_filenames = [f[:-2] + ".txt" for f in spike_absolute_filenames]

lfp_end = ".egf"

lfp_absolute_filenames = [f.split(".")[0] + lfp_end for f in spike_absolute_filenames]
print(lfp_absolute_filenames)

collection = NDataContainer(share_positions=False, load_on_fly=True)
collection.add_files(
    NDataContainer.EFileType.Spike, (spike_absolute_filenames, spike_names, None))
collection.add_files(
    NDataContainer.EFileType.LFP, lfp_absolute_filenames
)
collection.add_files(
    NDataContainer.EFileType.Position, position_absolute_filenames)

collection.setup()

['C:\\Users\\smartin5\\Recordings\\LCA1\\060319D_LCA1_linear_15min.egf', 'C:\\Users\\smartin5\\Recordings\\LCA1\\060319E_LCA1_linear_sleep_32min.egf']


In [5]:
# Setup the appropriate units
#collection.list_all_units()
# collection.set_units(["all", "all"])
run_units = [3, 2, 4] # top, middle, bottom is 2, 3, 4
sleep_units = [2, 3, 4, 6, 7, 8] # Units from powerpoint file
collection.set_units([run_units, sleep_units])

In [ ]:
# Spectrum plot
graph_data = collection.get_data(1).lfp.spectrum(
    window = 0.4, noverlap = 0.04, nfft = 2048, ptype = 'psd', 
    prefilt = True, filtset = [10, 100, 'highpass'], 
    fmax = 2400, db = False, tr = False, slice=slice(0, 9600, 1))
fig = nc_plot.lfp_spectrum(graph_data)

In [ ]:
# Displaying the filtered data
fs = collection.get_data(1).lfp.get_sampling_rate()
sample_start = int((924.4 * 4800))
sample_end = sample_start + 1200
lfp_samples = collection.get_data(1).get_samples()[sample_start:sample_end]
time = collection.get_data(1).lfp.get_timestamp()
filtered_lfp = butter_filter(lfp_samples, fs, 10, 100, 240, 'bandpass') # SWR
rms_data = window_rms(filtered_lfp, 33, "same")
ax, fig = nc_plot._make_ax_if_none(None)
ax.plot(time[sample_start:sample_end], filtered_lfp, color='r')
ax.plot(time[sample_start:sample_end], lfp_samples, color='b')
ax.plot(time[sample_start:sample_end], rms_data, color='g')

In [ ]:
results = nca.replay(collection, 0, 1)
fig = nc_plot.replay_summary(results)
plt.savefig("Nice_out.png")

Best assignment is {2: (4, 0.9669052702547893), 3: (8, 0.9903904138800321), 4: (7, 0.9516436207398975)}


In [ ]:
print(results.keys())

In [ ]:
swr_amt = len(results["swr times"])
lfp_amt = len(results["lfp samples"])
print(100 * swr_amt / lfp_amt)

In [ ]:
import math

def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

In [ ]:
# Look at areas of interest
from neurochat.nc_utils import get_axona_colours
unit_hist = results['mua histogram']
peaks2 = results['swr times']
sleep_samp = collection.subsample(1)
mua_indices = np.argwhere(unit_hist[0] == 3)
interest_ranges = []
for mua in [unit_hist[1][j] for j in mua_indices.flatten()]:
    # Is the centre lying in a SWR
    if any(
        mua - 0.5 <= peak - 0.4 <= mua + 0.5 or
        mua - 0.5 <= peak + 0.4 <= mua + 0.5
        for peak in peaks2
    ):
        interest_ranges.append((mua - 0.5, mua + 0.5)) # 1sec bins

        
# Plot the interest ranges
print(interest_ranges)
for i_range in interest_ranges:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4,7))
    nc_plot.spike_raster(
        nca.spike_times(sleep_samp, ranges=[i_range]),
        linewidths=1, ax=ax, colors=get_axona_colours()[:results["num cells"]],
        #xlim=(round(i_range[0], 1), round(i_range[1], 1)),
        orientation="vertical")
    plt.tight_layout()
    plt.savefig(str(i_range) + ".png")